In [29]:
import pandas as pd
import re
from urllib.parse import urlparse


df_user_reviews = pd.read_csv('UserReviews.txt', delimiter='\t', encoding='ISO-8859-1')
df_meta = pd.read_csv('metaClean43Brightspace.txt', delimiter='\t', encoding='ISO-8859-1')
df_MovieID = pd.read_csv('NewMovieTable.csv', encoding='ISO-8859-1')


df_reviews_only = df_user_reviews[df_user_reviews['Rev'].notna() & (df_user_reviews['Rev'].str.strip() != '')]


df_reviews_only = df_reviews_only.drop_duplicates()


columns_to_drop = [
    "WC", "Analytic", "Clout", "Authentic", "Tone", "WPS", "Sixltr", "Dic", 
    "function", "pronoun", "ppron", "i", "we", "you", "shehe", "they", "ipron", 
    "article", "prep", "auxverb", "adverb", "conj", "negate", "verb", "adj", 
    "compare", "interrog", "number", "quant", "affect", 
    "anx", "anger", "sad", "social", "family", "friend", "female", "male", 
    "cogproc", "insight", "cause", "discrep", "tentat", "certain", "differ", 
    "percept", "see", "hear", "feel", "bio", "body", "health", "sexual", 
    "ingest", "drives", "affiliation", "achieve", "power", "reward", "risk", 
    "focuspast", "focuspresent", "focusfuture", "relativ", "motion", "space", 
    "time", "work", "leisure", "home", "money", "relig", "death", "informal", 
    "swear", "netspeak", "assent", "nonflu", "filler", "AllPunc", "Period", 
    "Comma", "Colon", "SemiC", "QMark", "Exclam", "Dash", "Quote", "Apostro", 
    "Parenth", "OtherP", "dateP"
]
df_reviews_only = df_reviews_only.drop(columns=columns_to_drop, errors='ignore')


df_reviews_only.rename(columns={'thumbstot': 'totalthumbs'}, inplace=True)


df_reviews_only['reviewer'] = df_reviews_only['reviewer'].str.replace("'", "").str.replace(".", "").str.lower()


def extract_title_from_url(url):
    path = urlparse(url).path
    title = path.split('/')[-1]
    return title.replace('-', ' ').lower().strip()

df_reviews_only['extracted_title'] = df_reviews_only['url'].apply(extract_title_from_url)


def clean_text(text):
    if pd.isna(text):
        return ''
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_reviews_only['normalized_title'] = df_reviews_only['extracted_title'].apply(clean_text)
df_MovieID['normalized_title'] = df_MovieID['TITLE'].apply(clean_text)


df_reviews_only['review_id'] = range(1, len(df_reviews_only) + 1)


df_reviews_only = df_reviews_only.merge(df_MovieID[['normalized_title', 'Movie_ID']], on='normalized_title', how='left')


if 'ID' in df_reviews_only.columns:
    df_reviews_only.rename(columns={'ID': 'Movie_ID'}, inplace=True)


df_reviews_only['posemo'] = df_reviews_only['posemo'].astype(str).str.replace(',', '.').astype(float)
df_reviews_only['negemo'] = df_reviews_only['negemo'].astype(str).str.replace(',', '.').astype(float)


unmatched_reviews = df_reviews_only[df_reviews_only['Movie_ID'].isna()]
if not unmatched_reviews.empty:
    print("Unmatched user reviews:")
    print(unmatched_reviews[['url', 'normalized_title']])


df_reviews_only.to_csv('/Users/laiba/Documents/GitHub/gbm/cleaned_user_reviews.csv', index=False)


print("After merge, columns are:", df_reviews_only.columns)
print(df_reviews_only.head())


/var/folders/l3/5c9wkk49081bn1c_8j1b6yq00000gn/T/ipykernel_67356/2580579192.py:6: DtypeWarning: Columns (1,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_user_reviews = pd.read_csv('UserReviews.txt', delimiter='\t', encoding='ISO-8859-1')


After merge, columns are: Index(['url', 'idvscore', 'reviewer', 'Rev', 'thumbsUp', 'thumbsTot', 'posemo',
       'negemo', 'extracted_title', 'normalized_title', 'review_id',
       'Movie_ID'],
      dtype='object')
                                        url idvscore       reviewer  \
0  https://www.metacritic.com/movie/bronson      8.0   longbottom94   
1  https://www.metacritic.com/movie/bronson      9.0        martinb   
2  https://www.metacritic.com/movie/bronson     10.0         jaakko   
3  https://www.metacritic.com/movie/bronson      6.0          capor   
4  https://www.metacritic.com/movie/bronson      8.0        orwellb   

                                                 Rev thumbsUp thumbsTot  \
0   'Many have dismissed this film for not explor...      2.0       2.0   
1   'Anyone who doesn t like this movie simply ju...      0.0       1.0   
2   'Not sure what to think at this film at first...      1.0       1.0   
3   'Nicely portrayed but it lacks the elements t...    

In [31]:
print(df_reviews_only.columns)


Index(['url', 'idvscore', 'reviewer', 'Rev', 'thumbsUp', 'thumbsTot', 'posemo',
       'negemo', 'extracted_title', 'normalized_title', 'review_id',
       'Movie_ID'],
      dtype='object')


In [28]:
print(df_reviews_only[['thumbsTot', 'thumbsUp']].isna().sum())


thumbsTot    0
thumbsUp     0
dtype: int64


In [30]:
max_bigint = 9223372036854775807

# Rename 'thumbsTot' to 'totalthumbs' and 'thumbsUp' to 'thumbsup'
df_reviews_only.rename(columns={'thumbsTot': 'totalthumbs', 'thumbsUp': 'thumbsup'}, inplace=True)

# Ensure you're using the correct column names
df_reviews_only['totalthumbs'] = pd.to_numeric(df_reviews_only['totalthumbs'], errors='coerce')
df_reviews_only['thumbsup'] = pd.to_numeric(df_reviews_only['thumbsup'], errors='coerce')

# Apply modulo to ensure values stay within the bigint range
df_reviews_only['totalthumbs'] = df_reviews_only['totalthumbs'].apply(lambda x: x % max_bigint if pd.notna(x) else x)
df_reviews_only['thumbsup'] = df_reviews_only['thumbsup'].apply(lambda x: x % max_bigint if pd.notna(x) else x)
df_reviews_only['review_id'] = df_reviews_only['review_id'].apply(lambda x: x % max_bigint if pd.notna(x) else x)
df_reviews_only['Movie_ID'] = df_reviews_only['Movie_ID'].apply(lambda x: x % max_bigint if pd.notna(x) else x)

# Print the max values to check for outliers
print("Max Movie_ID:", df_reviews_only['Movie_ID'].max())
print("Max review_id:", df_reviews_only['review_id'].max())
print("Max totalthumbs:", df_reviews_only['totalthumbs'].max())
print("Max thumbsup:", df_reviews_only['thumbsup'].max())


KeyError: 'totalthumbs'

In [20]:
import psycopg2

# Step 1: Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="Laiba786",
    port="5432"
)
cursor = conn.cursor()

# Step 2: Create Tables if not exists
create_movie_table_query = """
CREATE TABLE IF NOT EXISTS movie_table (
    Movie_ID SERIAL PRIMARY KEY,
    Title TEXT UNIQUE,   -- Ensure title is unique
    WorldwideBoxOffice FLOAT
);
"""
create_user_reviews_query = """
CREATE TABLE IF NOT EXISTS user_reviews (
    review_id SERIAL PRIMARY KEY,
    reviewscore FLOAT,
    posemo FLOAT,
    negemo FLOAT,
    totalthumbs INTEGER,
    thumbsup INTEGER,
    rev TEXT,
    url TEXT,
    movie_id BIGINT REFERENCES movie_table (Movie_ID)
);
"""

cursor.execute(create_movie_table_query)
cursor.execute(create_user_reviews_query)
conn.commit()

# Step 3: Insert Queries
insert_movie_query = """
INSERT INTO movie_table (Title, WorldwideBoxOffice)
VALUES (%s, %s)
RETURNING Movie_ID;
"""
insert_user_reviews_query = """
INSERT INTO user_reviews (review_id, movie_id, reviewscore, posemo, negemo, totalthumbs, thumbsup, rev, url)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (review_id) DO NOTHING;
"""




# Insert Data into Tables
for index, row in df_reviews_only.iterrows():
    try:
        # Step 4a: Check if the movie already exists by title
        cursor.execute("SELECT Movie_ID FROM movie_table WHERE Title = %s", (row['extracted_title'],))
        existing_movie = cursor.fetchone()

        if existing_movie:
            movie_id = existing_movie[0]  # Get the existing Movie_ID
        else:
            # Insert the movie if it doesn't exist, returning Movie_ID
            cursor.execute(insert_movie_query, (row['extracted_title'], None))
            movie_id = cursor.fetchone()[0]  # Get the newly generated Movie_ID

        # Debug print before insertion
        print(f"Inserting review_id: {row['review_id']}, movie_id: {movie_id}, thumbsTot: {row['thumbsTot']}, thumbsUp: {row['thumbsUp']}")
        
        # Step 4b: Insert into user_reviews
        cursor.execute(insert_user_reviews_query, (
            row['review_id'], movie_id, row['idvscore'], row['posemo'], row['negemo'],
            row['thumbsTot'], row['thumbsUp'], row['Rev'], row['url']
        ))
        
    except Exception as e:
        print(f"Error on row {index}: {e}")
        break 
    

conn.commit()

# Step 6: Check the columns in user_reviews table
cursor.execute("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'user_reviews';
""")
columns_info = cursor.fetchall()
for column in columns_info:
    print(column)

# Step 7: Close the cursor and connection
cursor.close()
conn.close()

print("Data inserted and schema retrieved successfully!")


Inserting review_id: 1, movie_id: 20029, thumbsTot: 2.0, thumbsUp: 2.0
Inserting review_id: 2, movie_id: 20029, thumbsTot: 1.0, thumbsUp: 0.0
Inserting review_id: 3, movie_id: 20029, thumbsTot: 1.0, thumbsUp: 1.0
Inserting review_id: 4, movie_id: 20029, thumbsTot: 1.0, thumbsUp: 0.0
Inserting review_id: 5, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 6, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 7, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 8, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 9, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 10, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 11, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 12, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 13, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
Inserting review_id: 14, movie_id: 20029, thumbsTot: 0.0, thumbsUp: 0.0
I